In [60]:
from pathlib import Path 
import requests
#import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import date
import pandas as pd



In [61]:

sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm010483*",
    security_token="1B6xNEvzIe5uroF6nRgKGTSk",
)

In [62]:
results = sf.query_all(
    """
    select id,Paciente__r.IdentificationId__pc, padre__c,Paciente__c,Paciente__r.Habilitado__c,Programas_por_Empresas__r.Datos_Empresa__c,datos2__c,  Programas_por_Empresas__c,Programas_por_Empresas__r.Name,  Empleador__c  from Paciente_Programas__c where Id = 'a1Y8c00000C5asnEAB' 
    """
)
derttyuiop = pd.DataFrame(results["records"])
derttyuiop.to_excel(
        "F:/Empresas SF/reportes/aaaaaa.xlsx", index=False
    )


In [63]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,RUT_Empresa__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "RUT_Empresa__c",
    ]
]
Empresas_SF.shape

(7109, 8)

In [64]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(6815, 5)

In [65]:
#Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "RUT_Empresa__c":"Rut Empresa",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Rut Empresa",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A"
    ]
]

Programas_empresas.shape

(7115, 10)

fin empresas y comienzo de rescate de pacientes


In [66]:
#CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        'Phone'
        
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

(1650639, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.

In [67]:
# buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña",
    }
)
Paciente_Programas_SF.shape

(1211766, 4)

In [68]:
#Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c", 
)
empresas_unidas.shape

(1212074, 13)

In [70]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx",index=False)

In [71]:
empresas_unidas.head()

,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,None,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353


In [162]:
#pacientes_empresas22 = pd.merge(
    #left=Pacientes_SF,
    #right=empresas_unidas,
    #how="left",
    #left_on="Id Account Paciente",
    #right_on="Id Account", 
#)
#pacientes_empresas22 = pacientes_empresas22.drop_duplicates(['Rut'], keep='last')
#pacientes_empresas22 = pacientes_empresas22[
    #["Rut", "Nombre", "Nombre Campaña", "Nombre Empresa"]
#]


In [163]:
#pacientes_empresas22.shape

In [164]:
#apprix_1 = pacientes_empresas22.iloc[:848826,:]

#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts_habilitados1.xlsx",index=False)


In [165]:
#apprix_2 = pacientes_empresas22.iloc[848827:,:]#
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts_habilitados2.xlsx",index=False)

In [166]:
#pacientes_empresas22.head()

Carga de Archivo carga

In [72]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "MetLife"
Pais = "Chile"
#Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion


In [73]:
Pacientes_Archivo.head(10)

,poliza,Columna1,rut1,rutAsegurado,nombres,apellidos,sexo,nacimicfento,fdgrdfg,rethyerh,...,nacimiento,relacionConAsegurado,direccion,ciudad,email,celular,campanaPaciente,fecha_modificacion,empleador,Id Programas Por Empresa
0,340020207,340020207,26760594-7,26760594-7,EDUARDO,ASUAJE CAMACHO,Masculino,06-10-1995,6,10,...,1995-10-06,AS,NaN,NaN,NaN,977428255,Metlife,2023-04-22,Inmobiliaria Los Tres Antonios S. A.,a1W8c000007vDS3EAM
1,340007222,340007222,14194443-6,14194443-6,MARITZA DEL CAR,RETAMAL VILCHES,Femenino,03-03-1981,3,3,...,1981-03-03,AS,NaN,NaN,NaN,957681022,Metlife,2023-04-22,Adm. De Fondos De Pensiones Cuprum S.A.,a1W8c0000083qwdEAA
2,340022509,340022509,16926279-9,16926279-9,SEBASTIAN,FARIAS BARRAZA,Masculino,21-02-1988,21,2,...,1988-02-21,AS,NaN,NaN,NaN,86833201,Metlife,2023-04-22,Siemens Large Drives S.A.,a1W8c0000083qyEEAQ
3,340023173,340023173,16581317-0,16581317-0,LORENZO ALEXAND,FIGUEROA RIVERA,Masculino,14-08-1987,14,8,...,1987-08-14,AS,NaN,NaN,NaN,993669925,Metlife,2023-04-22,Emin. Ingeniería Y Construción S.A.,a1W8c0000083qqMEAQ
4,340007120,340007120,21219084-5,14112127-8,JEAN,MUNOZ CHACON,Masculino,19-01-2003,19,1,...,2003-01-19,HI,NaN,NaN,NaN,984684854,Metlife,2023-04-22,Inacal S.A.,a1W8c000007vDS2EAM
5,340007120,340007120,24375360-0,14112127-8,THIARE,MUNOZ CHACON,Femenino,10-09-2013,10,9,...,2013-09-10,HI,NaN,NaN,NaN,984684854,Metlife,2023-04-22,Inacal S.A.,a1W8c000007vDS2EAM
6,340007120,340007120,22233780-1,14112127-8,DARIO,MUNOZ CHACON,Masculino,11-10-2006,11,10,...,2006-10-11,HI,NaN,NaN,NaN,984684854,Metlife,2023-04-22,Inacal S.A.,a1W8c000007vDS2EAM
7,340007120,340007120,25656492-0,14112127-8,JESUS,MUNOZ CHACON,Masculino,30-01-2017,30,1,...,2017-01-30,HI,NaN,NaN,NaN,984684854,Metlife,2023-04-22,Inacal S.A.,a1W8c000007vDS2EAM
8,340023500,340023500,16662040-6,16662040-6,FABIAN GONZALO,BRIONES LOPEZ,Masculino,05-11-1987,5,11,...,1987-11-05,AS,NaN,NaN,NaN,944491738,Metlife,2023-04-22,Kushki Chile SPA,a1W8c0000083wcOEAQ
9,340023324,340023324,16494607-K,16494607-K,KEVIN FABRIZIO,MARTINEZ NUNEZ,Masculino,16-05-1987,16,5,...,1987-05-16,AS,NaN,NaN,NaN,NaN,Metlife,2023-04-22,Empresa De Montajes Industriales Salfa S.A.,a1W8c0000083wcqEAA


In [74]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)


In [75]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")

Pacientes_Archivo.head()

,poliza,Columna1,rut1,rutAsegurado,nombres,apellidos,sexo,nacimicfento,fdgrdfg,rethyerh,...,relacionConAsegurado,direccion,ciudad,email,celular,campanaPaciente,fecha_modificacion,empleador,Id Programas Por Empresa,rut
0,340020207,340020207,26760594-7,26760594-7,EDUARDO,ASUAJE CAMACHO,Masculino,06-10-1995,6,10,...,AS,NaN,NaN,NaN,977428255,Metlife,2023-04-22,Inmobiliaria Los Tres Antonios S. A.,a1W8c000007vDS3EAM,267605947
1,340007222,340007222,14194443-6,14194443-6,MARITZA DEL CAR,RETAMAL VILCHES,Femenino,03-03-1981,3,3,...,AS,NaN,NaN,NaN,957681022,Metlife,2023-04-22,Adm. De Fondos De Pensiones Cuprum S.A.,a1W8c0000083qwdEAA,141944436
2,340022509,340022509,16926279-9,16926279-9,SEBASTIAN,FARIAS BARRAZA,Masculino,21-02-1988,21,2,...,AS,NaN,NaN,NaN,86833201,Metlife,2023-04-22,Siemens Large Drives S.A.,a1W8c0000083qyEEAQ,169262799
3,340023173,340023173,16581317-0,16581317-0,LORENZO ALEXAND,FIGUEROA RIVERA,Masculino,14-08-1987,14,8,...,AS,NaN,NaN,NaN,993669925,Metlife,2023-04-22,Emin. Ingeniería Y Construción S.A.,a1W8c0000083qqMEAQ,165813170
4,340007120,340007120,21219084-5,14112127-8,JEAN,MUNOZ CHACON,Masculino,19-01-2003,19,1,...,HI,NaN,NaN,NaN,984684854,Metlife,2023-04-22,Inacal S.A.,a1W8c000007vDS2EAM,212190845


In [76]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()

Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_19332\1949753364.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")


,poliza,Columna1,rut1,rutAsegurado,nombres,apellidos,sexo,nacimicfento,fdgrdfg,rethyerh,...,relacionConAsegurado,direccion,ciudad,email,celular,campanaPaciente,fecha_modificacion,empleador,Id Programas Por Empresa,rut
0,340020207,340020207,26760594-7,26760594-7,EDUARDO,ASUAJE CAMACHO,Masculino,06-10-1995,6,10,...,AS,NaN,NaN,NaN,977428255,Metlife,2023-04-22,Inmobiliaria Los Tres Antonios S. A.,a1W8c000007vDS3EAM,267605947
1,340007222,340007222,14194443-6,14194443-6,MARITZA DEL CAR,RETAMAL VILCHES,Femenino,03-03-1981,3,3,...,AS,NaN,NaN,NaN,957681022,Metlife,2023-04-22,Adm. De Fondos De Pensiones Cuprum S.A.,a1W8c0000083qwdEAA,141944436
2,340022509,340022509,16926279-9,16926279-9,SEBASTIAN,FARIAS BARRAZA,Masculino,21-02-1988,21,2,...,AS,NaN,NaN,NaN,86833201,Metlife,2023-04-22,Siemens Large Drives S.A.,a1W8c0000083qyEEAQ,169262799
3,340023173,340023173,16581317-0,16581317-0,LORENZO ALEXAND,FIGUEROA RIVERA,Masculino,14-08-1987,14,8,...,AS,NaN,NaN,NaN,993669925,Metlife,2023-04-22,Emin. Ingeniería Y Construción S.A.,a1W8c0000083qqMEAQ,165813170
4,340007120,340007120,21219084-5,14112127-8,JEAN,MUNOZ CHACON,Masculino,19-01-2003,19,1,...,HI,NaN,NaN,NaN,984684854,Metlife,2023-04-22,Inacal S.A.,a1W8c000007vDS2EAM,212190845


In [77]:
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
# Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()

Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("-", "")


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(Pacientes_SF["Rut"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_19332\1486029825.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_19332\1486029825.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [78]:
#aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
Pacientes_con_id.shape

(6919, 32)

In [79]:
Pacientes_con_id.head()

,poliza,Columna1,rut1,rutAsegurado,nombres,apellidos,sexo,nacimicfento,fdgrdfg,rethyerh,...,RutEstaoNo,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,340020207,340020207,26760594-7,26760594-7,Eduardo,Asuaje Camacho,Masculino,06-10-1995,6,10,...,True,0018c00002l65hFAAQ,267605947,Eduardo Asuaje Camacho,RUT,None,None,None,Hombre,977428255
1,340007222,340007222,14194443-6,14194443-6,Maritza Del Car,Retamal Vilches,Femenino,03-03-1981,3,3,...,True,0018c00002eVwOLAA0,141944436,Maritza Del Carmen Retamal Vilches,RUT,None,None,None,Hombre,None
2,340022509,340022509,16926279-9,16926279-9,Sebastian,Farias Barraza,Masculino,21-02-1988,21,2,...,True,0018c00002axb0WAAQ,169262799,Farias Barraza Sebastian Hernan,RUT,None,None,None,Hombre,None
3,340023173,340023173,16581317-0,16581317-0,Lorenzo Alexand,Figueroa Rivera,Masculino,14-08-1987,14,8,...,True,0018c00002l65hGAAQ,165813170,Lorenzo Alexand Figueroa Rivera,RUT,None,None,None,Hombre,993669925
4,340007120,340007120,21219084-5,14112127-8,Jean,Munoz Chacon,Masculino,19-01-2003,19,1,...,True,0018c00002l65hHAAQ,212190845,Jean Munoz Chacon,RUT,None,None,None,Hombre,984684854


In [80]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = (
        Pacientes_con_id["rut"].astype(str).str.upper()
    )
    Subir_pacientes.head(20)
if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = (
        Pacientes_con_id["email"].astype(str).str.lower().replace("nan", "")
    )
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"]

if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "M": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "O",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "F ": "Mujer",
        "Varón": "Hombre",
        "varon":"Hombre"
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = (
        Pacientes_con_id["celular"].apply(str).str.strip().replace("+", "")
    )
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.title().replace("NaN", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.title().replace("Nan", "")

Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"


if Pais == "Chile":
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
else:
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "CURP"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtsEAE"


Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account.xlsx", index=False)
Subir_pacientes.head()

,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonEmail,PersonBirthdate,HealthCloudGA__Gender__pc,BillingStreet,BillingCity,Phone,RecordTypeId,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
6875,False,Zingara,Alvarez Cruz,115081233,,1969-12-22,Mujer,NaN,NaN,992265806,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
6876,False,Martin,Matthews Alvarez,214727552,,2003-12-26,Hombre,NaN,NaN,992265806,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
6877,False,Catalina,Cifuentes Parra,17833297K,,1991-02-12,Mujer,NaN,NaN,965954065,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
6878,False,Gael,Riquelme Cifuentes,280610488,,2023-02-05,Hombre,NaN,NaN,965954065,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
6882,False,Valentina,Cotre Aguilera,225062730,,2008-09-20,Mujer,NaN,NaN,954956565,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE


In [81]:
clinica_virtual = pd.DataFrame()
clinica_virtual["rut"] = Pacientes_con_id["rut1"].str.upper().str.strip()
clinica_virtual["firstName"] = Pacientes_con_id["nombres"].str.title().str.strip()
clinica_virtual["lastName"] = Pacientes_con_id["apellidos"].str.title().str.strip()
clinica_virtual["accountId"] = campaña


Codigos_Clinica_Virtual = {
    "BCI": "1",
    "Farmacias Ahumada": "2",
    "MetLife": "3",
    "Vida Security": "4",
    "Willis": "5",
    "Entel": "6",
    "Testing": "7",
    "3SP": "8",
    "HomeCare": "9",
    "Pedro de Valdivia": "10",
    "Colaboradores": "11",
    "MineraAbra": "12",
    "CMPC": "13",
    "Lipigas": "14",
    "Asismed": "15",
    "Siempre Perú": "16",
    "Farmacias Cruz Verde": "17",
    "WFS Food Services": "18",
    "VidaCel": "19",
    "Dermik": "20",
    "EY": "21",
    "IND": "22",
    "Sindicato Walmart": "23",
    "Instruvalve": "24",
    "VidaCel Colaboradores": "25",
    "Socofar": "26",
    "Milan Fajanovic": "27",
    "THBseguros": "28",
    "Pesco": "29",
    "Burgos - Consorcio": "30",
    "COFAMA-SILENTIUM": "31",
    "KAUFMANN": "32",
    "QuePlan": "33",
    "Zurich": "34",
    "Colaboradores Siempre Chile": "35",
    "Talo": "36",
    "Transvip": "37",
    "Vida Security Colaboradores": "38",
    "FORUS": "39",
    "Sura": "40",
    "Southbrige": "41",
    "Construmart": "42",
    "BICE Vida": "43",
    "Lockton": "44",
    "Mercer": "48",
    "Grupo PSG": "46",
    "Siempre Chile": "47",
}

clinica_virtual.replace({"accountId": Codigos_Clinica_Virtual}, inplace=True)
clinica_virtual.to_excel("F:/Empresas SF/reportes/Clinica_Virtual.xlsx", index=False)

In [82]:
campaña_seleccionada = empresas_unidas[empresas_unidas["Campaña"] == campaña]
campaña_seleccionada.head()


,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,None,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353


In [83]:
campaña_seleccionada.shape

(749091, 13)

In [84]:
if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )

In [85]:
campaña_en_pacientes = pd.merge(
    left=Pacientes_con_id,
    right=campaña_seleccionada,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)
if ("poliza" in campaña_en_pacientes.columns) == True:
    campaña_en_pacientes["poliza"] = campaña_en_pacientes["poliza"].astype(str)

campaña_en_pacientes = campaña_en_pacientes.rename(
    columns={"Programas_por_Empresas__c": "programas por empresa"}
)
campaña_en_pacientes["Paciente__c"] = campaña_en_pacientes["Id Account Paciente"]
campaña_en_pacientes["Programas_por_Empresas__c"] = campaña_en_pacientes[
    "programas por empresa"
]
campaña_en_pacientes.to_excel(
    "F:/Empresas SF/reportes/archivo_campaña2.xlsx", index=False
)

In [86]:

#apprix_1 = Pacientes_SF.iloc[:790552,:]
#apprix_2 = Pacientes_SF.iloc[790553:,:]
#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)
